In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import yaml
import copy

In [2]:
filename = r'.\data\data_1\data\0000000150.png'
camera_file =  r'.\data\data_1\camera_params.yaml'
with open(camera_file) as file:
    camera_coeffs = yaml.load(file, Loader=yaml.FullLoader)
K = np.array(list(float(i) for i in (camera_coeffs['K']).split())).reshape(3,3)
D = np.array(list(float(i) for i in (camera_coeffs['D']).split()))
C = cv2.VideoCapture('challenge_video.mp4')


In [3]:
def homography(img1, img2):
    ind = 0
    A_matrix = np.empty((8, 9))
    
    for pixel in range(0, len(img1)):
        
        x_1 = img1[pixel][0]
        y_1 = img1[pixel][1]

        x_2 = img2[pixel][0]
        y_2 = img2[pixel][1]

        A_matrix[ind] = np.array([x_1, y_1, 1, 0, 0, 0, -x_2*x_1, -x_2*y_1, -x_2])
        A_matrix[ind + 1] = np.array([0, 0, 0, x_1, y_1, 1, -y_2*x_1, -y_2*y_1, -y_2])

        ind = ind + 2
    
    U, s, V = np.linalg.svd(A_matrix, full_matrices=True)
    V = (copy.deepcopy(V)) / (copy.deepcopy(V[8][8]))
    H = V[8,:].reshape(3, 3)
    return H


In [4]:
while(True):
    ret, image = C.read()
    if ret == True:
        undistorted = cv2.undistort(image, K, D)
        #gray = cv2.cvtColor(undistorted,cv2.COLOR_BGR2GRAY)
        gauss = cv2.GaussianBlur(undistorted,(7,7),1)
        destination_1 = np.array([[0, 0],[300, 0],[0,300],[300, 300]], dtype = "float32")
        source_1 = np.array([[560,450],[740, 450],[95, 710],[1260, 710]], dtype = "float32")
        H = homography(source_1, destination_1)
# Hinv = np.linalg.inv(H)
        warp = cv2.warpPerspective(gauss, H, (255, 255))
        channel = warp[:,:,2]               #Selecting the red channel of the image
        ret, thresh = cv2.threshold(channel, 110, 255, cv2.THRESH_BINARY)               #Thresholding the image
        sobelx = cv2.Sobel(thresh, cv2.CV_64F, 1, 0)
        sobely = cv2.Sobel(thresh, cv2.CV_64F, 0, 1)
        sobel = np.sqrt((sobelx**2) + (sobely**2)) 
#         plt.imshow(sobel)
        cv2.imshow('sobel', sobel)
        k = cv2.waitKey(1)
        if k == 27:
            break
        else:
            break

C.release()        
# cv2.destroyAllWindows()








